<a href="https://colab.research.google.com/github/larsmoan/COMP3710/blob/main/assignement_2/cnn/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Part two of the assignement